In [1]:
########################## 포스트 일괄 등록 - 최신순 100개 ##########################

import requests
import json
import time
from google.oauth2 import service_account
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError




# 워드프레스 설정
WORDPRESS_URL = "https://prayer-church.co.kr/"
WORDPRESS_API_URL = f"{WORDPRESS_URL}/wp-json/wp/v2"
POSTS_PER_PAGE = 100  # 한 번에 가져올 포스트 수 (최대 100)

# 구글 인덱싱 API 설정
SERVICE_ACCOUNT_FILE = "/Users/a/Desktop/Work/Blog/2024 Adsense/prompt/black-outlet-426702-n1-98f06232b595.json"  # 서비스 계정 인증 파일

def get_wordpress_posts(page=1):
    """워드프레스 API에서 게시물을 가져옵니다."""
    try:
        response = requests.get(
            f"{WORDPRESS_API_URL}/posts",
            params={
                "page": page,
                "per_page": POSTS_PER_PAGE,
                "status": "publish",
                "_fields": "id,link"  # 필요한 필드만 요청
            }
        )
        response.raise_for_status()
        
        # 전체 페이지 수 확인
        total_pages = int(response.headers.get("X-WP-TotalPages", 1))
        
        return response.json(), total_pages
    except requests.exceptions.RequestException as e:
        print(f"워드프레스 API 요청 오류: {e}")
        return [], 0

def get_all_wordpress_posts():
    """모든 워드프레스 포스트 URL을 수집합니다."""
    all_posts = []
    page = 1
    total_pages = 1
    
    while page <= total_pages:
        posts, total_pages = get_wordpress_posts(page)
        if not posts:
            break
        
        all_posts.extend(posts)
        print(f"페이지 {page}/{total_pages} 처리 완료, {len(posts)}개 포스트 수집")
        page += 1
        time.sleep(1)  # API 호출 제한 방지를 위한 딜레이
    
    return all_posts

def authenticate_indexing_api():
    """구글 Indexing API 인증"""
    try:
        credentials = service_account.Credentials.from_service_account_file(
            SERVICE_ACCOUNT_FILE,
            scopes=['https://www.googleapis.com/auth/indexing']
        )
        service = build('indexing', 'v3', credentials=credentials)
        return service
    except Exception as e:
        print(f"인덱싱 API 인증 오류: {e}")
        return None

def submit_url_to_index(service, url):
    """인덱싱 API에 URL 인덱싱 요청"""
    try:
        response = service.urlNotifications().publish(
            body={
                'url': url,
                'type': 'URL_UPDATED'  # 콘텐츠 업데이트 알림
            }
        ).execute()
        print(f"URL 제출 성공: {url}")
        return True
    except HttpError as e:
        print(f"URL 제출 실패 ({url}): {e}")
        return False

def main():
    # 1. 워드프레스에서 모든 포스트 가져오기
    print("워드프레스 포스트 수집 중...")
    posts = get_all_wordpress_posts()
    
    if not posts:
        print("포스트를 찾을 수 없습니다.")
        return
    
    print(f"총 {len(posts)}개의 포스트를 찾았습니다.")
    
    # 2. 인덱싱 API 인증
    print("구글 인덱싱 API 인증 중...")
    service = authenticate_indexing_api()
    
    if not service:
        print("인덱싱 API 인증에 실패했습니다.")
        return
    
    # 3. 각 URL을 인덱싱 API에 제출
    print("URL 인덱싱 제출 시작...")
    success_count = 0
    
    for post in posts:
        post_url = post.get('link')
        if post_url:
            if submit_url_to_index(service, post_url):
                success_count += 1
            time.sleep(1)  # API 호출 제한 방지를 위한 딜레이
    
    print(f"인덱싱 요청 완료: 총 {len(posts)}개 중 {success_count}개 성공")

if __name__ == "__main__":
    main()

워드프레스 포스트 수집 중...
페이지 1/5 처리 완료, 100개 포스트 수집
페이지 2/5 처리 완료, 100개 포스트 수집
페이지 3/5 처리 완료, 100개 포스트 수집
페이지 4/5 처리 완료, 100개 포스트 수집
페이지 5/5 처리 완료, 28개 포스트 수집
총 428개의 포스트를 찾았습니다.
구글 인덱싱 API 인증 중...
URL 인덱싱 제출 시작...
URL 제출 성공: https://prayer-church.co.kr/%ec%a3%bc%ec%9d%bc-%eb%82%ae%ec%98%88%eb%b0%b0-%eb%8c%80%ed%91%9c-%ea%b8%b0%eb%8f%84%eb%ac%b8-10%ec%9b%94/
URL 제출 성공: https://prayer-church.co.kr/%ec%a3%bc%ec%9d%bc-%eb%82%ae%ec%98%88%eb%b0%b0-%eb%8c%80%ed%91%9c-%ea%b8%b0%eb%8f%84%eb%ac%b8-9%ec%9b%94/
URL 제출 성공: https://prayer-church.co.kr/%ec%a3%bc%ec%9d%bc-%eb%82%ae%ec%98%88%eb%b0%b0-%eb%8c%80%ed%91%9c-%ea%b8%b0%eb%8f%84%eb%ac%b8-8%ec%9b%94/
URL 제출 성공: https://prayer-church.co.kr/%ec%a3%bc%ec%9d%bc-%eb%82%ae%ec%98%88%eb%b0%b0-%eb%8c%80%ed%91%9c-%ea%b8%b0%eb%8f%84%eb%ac%b8-7%ec%9b%94/
URL 제출 성공: https://prayer-church.co.kr/%ec%a3%bc%ec%9d%bc-%eb%82%ae%ec%98%88%eb%b0%b0-%eb%8c%80%ed%91%9c-%ea%b8%b0%eb%8f%84%eb%ac%b8-6%ec%9b%94/
URL 제출 성공: https://prayer-church.co.kr/%ec%a3%bc%ec%9d%bc-%eb%82%a

URL 제출 성공: https://prayer-church.co.kr/%ec%83%88%eb%b2%bd%ec%98%88%eb%b0%b0-%eb%8c%80%ed%91%9c%ea%b8%b0%eb%8f%84%eb%ac%b8-10%ec%9b%94-%eb%84%b7%ec%a7%b8%ec%a3%bc/
URL 제출 성공: https://prayer-church.co.kr/%ec%83%88%eb%b2%bd%ec%98%88%eb%b0%b0-%eb%8c%80%ed%91%9c%ea%b8%b0%eb%8f%84%eb%ac%b8-10%ec%9b%94-%ec%85%8b%ec%a7%b8%ec%a3%bc/
URL 제출 성공: https://prayer-church.co.kr/%ec%83%88%eb%b2%bd%ec%98%88%eb%b0%b0-%eb%8c%80%ed%91%9c%ea%b8%b0%eb%8f%84%eb%ac%b8-10%ec%9b%94-%eb%91%98%ec%a7%b8%ec%a3%bc/
URL 제출 성공: https://prayer-church.co.kr/%ec%83%88%eb%b2%bd%ec%98%88%eb%b0%b0-%eb%8c%80%ed%91%9c%ea%b8%b0%eb%8f%84%eb%ac%b8-10%ec%9b%94-%ec%b2%ab%ec%a7%b8%ec%a3%bc/
URL 제출 성공: https://prayer-church.co.kr/%ec%83%88%eb%b2%bd%ec%98%88%eb%b0%b0-%eb%8c%80%ed%91%9c%ea%b8%b0%eb%8f%84%eb%ac%b8-9%ec%9b%94-%eb%a7%88%ec%a7%80%eb%a7%89%ec%a3%bc/
URL 제출 성공: https://prayer-church.co.kr/%ec%83%88%eb%b2%bd%ec%98%88%eb%b0%b0-%eb%8c%80%ed%91%9c%ea%b8%b0%eb%8f%84%eb%ac%b8-9%ec%9b%94-%eb%84%b7%ec%a7%b8%ec%a3%bc/
URL 제출 성공: http

URL 제출 성공: https://prayer-church.co.kr/%ec%88%98%ec%9a%94%ec%98%88%eb%b0%b0-%eb%8c%80%ed%91%9c%ea%b8%b0%eb%8f%84%eb%ac%b8-10%ec%9b%94-%eb%91%98%ec%a7%b8%ec%a3%bc/
URL 제출 성공: https://prayer-church.co.kr/%ec%88%98%ec%9a%94%ec%98%88%eb%b0%b0-%eb%8c%80%ed%91%9c%ea%b8%b0%eb%8f%84%eb%ac%b8-10%ec%9b%94-%ec%b2%ab%ec%a7%b8%ec%a3%bc/
URL 제출 성공: https://prayer-church.co.kr/%ec%88%98%ec%9a%94%ec%98%88%eb%b0%b0-%eb%8c%80%ed%91%9c%ea%b8%b0%eb%8f%84%eb%ac%b8-9%ec%9b%94-%eb%a7%88%ec%a7%80%eb%a7%89%ec%a3%bc/
URL 제출 성공: https://prayer-church.co.kr/%ec%88%98%ec%9a%94%ec%98%88%eb%b0%b0-%eb%8c%80%ed%91%9c%ea%b8%b0%eb%8f%84%eb%ac%b8-9%ec%9b%94-%eb%84%b7%ec%a7%b8%ec%a3%bc/
URL 제출 성공: https://prayer-church.co.kr/%ec%88%98%ec%9a%94%ec%98%88%eb%b0%b0-%eb%8c%80%ed%91%9c%ea%b8%b0%eb%8f%84%eb%ac%b8-9%ec%9b%94-%ec%85%8b%ec%a7%b8%ec%a3%bc/
URL 제출 성공: https://prayer-church.co.kr/%ec%88%98%ec%9a%94%ec%98%88%eb%b0%b0-%eb%8c%80%ed%91%9c%ea%b8%b0%eb%8f%84%eb%ac%b8-9%ec%9b%94-%eb%91%98%ec%a7%b8%ec%a3%bc/
URL 제출 성공: https:

URL 제출 성공: https://prayer-church.co.kr/%ec%9c%a1%ec%8b%a0%ec%9d%b4-%ec%a7%80%ec%b9%9c-%ec%9e%90%eb%85%80%eb%a5%bc-%ec%9c%84%ed%95%9c-%ea%b8%b0%eb%8f%84%eb%ac%b8/
URL 제출 성공: https://prayer-church.co.kr/%ec%83%81%ec%b2%98%eb%a5%bc-%eb%b0%9b%ec%9d%80-%ec%9e%90%eb%85%80%eb%a5%bc-%ec%9c%84%ed%95%9c-%ea%b8%b0%eb%8f%84%eb%ac%b8/
URL 제출 성공: https://prayer-church.co.kr/%ec%9e%90%eb%85%80%ec%9d%98-%ec%8b%9c%ed%97%98%ec%9d%84-%ec%9c%84%ed%95%9c-1%eb%b6%84-%ea%b8%b0%eb%8f%84%eb%ac%b8/
URL 제출 성공: https://prayer-church.co.kr/%ec%88%98%eb%8a%a5-%ec%9e%90%eb%85%80%eb%a5%bc-%ec%9c%84%ed%95%9c-1%eb%b6%84-%ea%b8%b0%eb%8f%84%eb%ac%b8/
URL 제출 성공: https://prayer-church.co.kr/%ec%a7%80%ed%98%9c%eb%a5%bc-%ec%9c%84%ed%95%9c-1%eb%b6%84-%ea%b8%b0%eb%8f%84%eb%ac%b8/
URL 제출 성공: https://prayer-church.co.kr/1%eb%b6%84-%ea%b8%b0%eb%8f%84-%ec%9e%90%eb%85%80%ec%9d%98-%eb%81%88%ea%b8%b0-%ea%b8%b0%eb%8f%84%eb%ac%b8/
URL 제출 성공: https://prayer-church.co.kr/1%eb%b6%84-%ea%b8%b0%eb%8f%84-%ec%9e%90%eb%85%80%ec%9d%98-%ec%84%b1

URL 제출 실패 (https://prayer-church.co.kr/%eb%a7%a4%ec%9d%bc-%ea%b8%b0%eb%8f%84%eb%ac%b8/): <HttpError 429 when requesting https://indexing.googleapis.com/v3/urlNotifications:publish?alt=json returned "Quota exceeded for quota metric 'Publish requests' and limit 'Publish requests per day' of service 'indexing.googleapis.com' for consumer 'project_number:123230913857'.". Details: "[{'@type': 'type.googleapis.com/google.rpc.ErrorInfo', 'reason': 'RATE_LIMIT_EXCEEDED', 'domain': 'googleapis.com', 'metadata': {'consumer': 'projects/123230913857', 'quota_unit': '1/d/{project}', 'quota_limit_value': '200', 'service': 'indexing.googleapis.com', 'quota_metric': 'indexing.googleapis.com/v3_publish_requests', 'quota_location': 'global', 'quota_limit': 'DefaultPublishRequestsPerDayPerProject'}}, {'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Request a higher quota limit.', 'url': 'https://cloud.google.com/docs/quotas/help/request_increase'}]}]">
URL 제출 실패 (https://prayer

...

URL 제출 실패 (https://prayer-church.co.kr/%ec%a7%a7%ec%9d%80-%eb%8c%80%ed%91%9c%ea%b8%b0%eb%8f%84%eb%ac%b8-%eb%aa%a8%ec%9d%8c-11%ec%9b%94/): <HttpError 429 when requesting https://indexing.googleapis.com/v3/urlNotifications:publish?alt=json returned "Quota exceeded for quota metric 'Publish requests' and limit 'Publish requests per day' of service 'indexing.googleapis.com' for consumer 'project_number:123230913857'.". Details: "[{'@type': 'type.googleapis.com/google.rpc.ErrorInfo', 'reason': 'RATE_LIMIT_EXCEEDED', 'domain': 'googleapis.com', 'metadata': {'service': 'indexing.googleapis.com', 'quota_location': 'global', 'quota_limit_value': '200', 'quota_limit': 'DefaultPublishRequestsPerDayPerProject', 'quota_metric': 'indexing.googleapis.com/v3_publish_requests', 'quota_unit': '1/d/{project}', 'consumer': 'projects/123230913857'}}, {'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Request a higher quota limit.', 'url': 'https://cloud.google.com/docs/quotas/help/

URL 제출 실패 (https://prayer-church.co.kr/%ec%a7%a7%ec%9d%80-%eb%8c%80%ed%91%9c-%ea%b8%b0%eb%8f%84-2025/): <HttpError 429 when requesting https://indexing.googleapis.com/v3/urlNotifications:publish?alt=json returned "Quota exceeded for quota metric 'Publish requests' and limit 'Publish requests per day' of service 'indexing.googleapis.com' for consumer 'project_number:123230913857'.". Details: "[{'@type': 'type.googleapis.com/google.rpc.ErrorInfo', 'reason': 'RATE_LIMIT_EXCEEDED', 'domain': 'googleapis.com', 'metadata': {'quota_metric': 'indexing.googleapis.com/v3_publish_requests', 'quota_unit': '1/d/{project}', 'quota_limit': 'DefaultPublishRequestsPerDayPerProject', 'quota_location': 'global', 'quota_limit_value': '200', 'consumer': 'projects/123230913857', 'service': 'indexing.googleapis.com'}}, {'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Request a higher quota limit.', 'url': 'https://cloud.google.com/docs/quotas/help/request_increase'}]}]">
URL 제출 실패 

URL 제출 실패 (https://prayer-church.co.kr/6%ec%9b%94-%ec%b2%ab%ec%a7%b8-%ec%a3%bc%ec%9d%bc-%eb%8c%80%ed%91%9c-%ea%b8%b0%eb%8f%84%eb%ac%b8/): <HttpError 429 when requesting https://indexing.googleapis.com/v3/urlNotifications:publish?alt=json returned "Quota exceeded for quota metric 'Publish requests' and limit 'Publish requests per day' of service 'indexing.googleapis.com' for consumer 'project_number:123230913857'.". Details: "[{'@type': 'type.googleapis.com/google.rpc.ErrorInfo', 'reason': 'RATE_LIMIT_EXCEEDED', 'domain': 'googleapis.com', 'metadata': {'quota_metric': 'indexing.googleapis.com/v3_publish_requests', 'consumer': 'projects/123230913857', 'quota_limit': 'DefaultPublishRequestsPerDayPerProject', 'quota_limit_value': '200', 'quota_unit': '1/d/{project}', 'quota_location': 'global', 'service': 'indexing.googleapis.com'}}, {'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Request a higher quota limit.', 'url': 'https://cloud.google.com/docs/quotas/help/

URL 제출 실패 (https://prayer-church.co.kr/%ec%a3%bc%ec%9d%bc-%ec%98%88%eb%b0%b0-%eb%8c%80%ed%91%9c-%ea%b8%b0%eb%8f%84%eb%ac%b8-100%ea%b0%9c/): <HttpError 429 when requesting https://indexing.googleapis.com/v3/urlNotifications:publish?alt=json returned "Quota exceeded for quota metric 'Publish requests' and limit 'Publish requests per day' of service 'indexing.googleapis.com' for consumer 'project_number:123230913857'.". Details: "[{'@type': 'type.googleapis.com/google.rpc.ErrorInfo', 'reason': 'RATE_LIMIT_EXCEEDED', 'domain': 'googleapis.com', 'metadata': {'quota_metric': 'indexing.googleapis.com/v3_publish_requests', 'consumer': 'projects/123230913857', 'quota_location': 'global', 'quota_limit_value': '200', 'quota_limit': 'DefaultPublishRequestsPerDayPerProject', 'quota_unit': '1/d/{project}', 'service': 'indexing.googleapis.com'}}, {'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Request a higher quota limit.', 'url': 'https://cloud.google.com/docs/quotas/hel

인덱싱 요청 완료: 총 428개 중 206개 성공


In [2]:
########################## 페이지 일괄 등록 - 최신순 100개 ##########################

import requests
import json
import time
from google.oauth2 import service_account
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# 워드프레스 설정
WORDPRESS_URL = "https://prayer-church.co.kr"  # 실제 워드프레스 사이트 URL
WORDPRESS_API_URL = f"{WORDPRESS_URL}/wp-json/wp/v2"
PAGES_PER_REQUEST = 100  # 한 번에 가져올 페이지 수 (최대 100)

# 구글 인덱싱 API 설정
SERVICE_ACCOUNT_FILE = "/Users/a/Desktop/Work/Blog/2024 Adsense/prompt/black-outlet-426702-n1-98f06232b595.json"  # 서비스 계정 인증 파일

def get_wordpress_pages(page_number=1):
    """워드프레스 API에서 페이지를 가져옵니다."""
    try:
        response = requests.get(
            f"{WORDPRESS_API_URL}/pages",  # 포스트 대신 페이지 엔드포인트 사용
            params={
                "page": page_number,
                "per_page": PAGES_PER_REQUEST,
                "status": "publish",
                "_fields": "id,link"  # 필요한 필드만 요청
            }
        )
        response.raise_for_status()
        
        # 전체 페이지 수 확인
        total_pages = int(response.headers.get("X-WP-TotalPages", 1))
        
        return response.json(), total_pages
    except requests.exceptions.RequestException as e:
        print(f"워드프레스 API 요청 오류: {e}")
        return [], 0

def get_all_wordpress_pages():
    """모든 워드프레스 페이지 URL을 수집합니다."""
    all_pages = []
    page_number = 1
    total_page_numbers = 1
    
    while page_number <= total_page_numbers:
        pages, total_page_numbers = get_wordpress_pages(page_number)
        if not pages:
            break
        
        all_pages.extend(pages)
        print(f"페이지 목록 {page_number}/{total_page_numbers} 처리 완료, {len(pages)}개 페이지 수집")
        page_number += 1
        time.sleep(1)  # API 호출 제한 방지를 위한 딜레이
    
    return all_pages

def authenticate_indexing_api():
    """구글 Indexing API 인증"""
    try:
        credentials = service_account.Credentials.from_service_account_file(
            SERVICE_ACCOUNT_FILE,
            scopes=['https://www.googleapis.com/auth/indexing']
        )
        service = build('indexing', 'v3', credentials=credentials)
        return service
    except Exception as e:
        print(f"인덱싱 API 인증 오류: {e}")
        return None

def submit_url_to_index(service, url):
    """인덱싱 API에 URL 인덱싱 요청"""
    try:
        response = service.urlNotifications().publish(
            body={
                'url': url,
                'type': 'URL_UPDATED'  # 콘텐츠 업데이트 알림
            }
        ).execute()
        print(f"URL 제출 성공: {url}")
        return True
    except HttpError as e:
        print(f"URL 제출 실패 ({url}): {e}")
        return False

def main():
    # 1. 워드프레스에서 모든 페이지 가져오기
    print("워드프레스 페이지 수집 중...")
    pages = get_all_wordpress_pages()
    
    if not pages:
        print("페이지를 찾을 수 없습니다.")
        return
    
    print(f"총 {len(pages)}개의 페이지를 찾았습니다.")
    
    # 2. 인덱싱 API 인증
    print("구글 인덱싱 API 인증 중...")
    service = authenticate_indexing_api()
    
    if not service:
        print("인덱싱 API 인증에 실패했습니다.")
        return
    
    # 3. 각 URL을 인덱싱 API에 제출
    print("URL 인덱싱 제출 시작...")
    success_count = 0
    
    for page in pages:
        page_url = page.get('link')
        if page_url:
            if submit_url_to_index(service, page_url):
                success_count += 1
            time.sleep(1)  # API 호출 제한 방지를 위한 딜레이
    
    print(f"인덱싱 요청 완료: 총 {len(pages)}개 중 {success_count}개 성공")

if __name__ == "__main__":
    main()

워드프레스 페이지 수집 중...
페이지 목록 1/1 처리 완료, 82개 페이지 수집
총 82개의 페이지를 찾았습니다.
구글 인덱싱 API 인증 중...
URL 인덱싱 제출 시작...
URL 제출 실패 (https://prayer-church.co.kr/prayers-short/): <HttpError 429 when requesting https://indexing.googleapis.com/v3/urlNotifications:publish?alt=json returned "Quota exceeded for quota metric 'Publish requests' and limit 'Publish requests per day' of service 'indexing.googleapis.com' for consumer 'project_number:123230913857'.". Details: "[{'@type': 'type.googleapis.com/google.rpc.ErrorInfo', 'reason': 'RATE_LIMIT_EXCEEDED', 'domain': 'googleapis.com', 'metadata': {'service': 'indexing.googleapis.com', 'consumer': 'projects/123230913857', 'quota_metric': 'indexing.googleapis.com/v3_publish_requests', 'quota_location': 'global', 'quota_limit_value': '200', 'quota_limit': 'DefaultPublishRequestsPerDayPerProject', 'quota_unit': '1/d/{project}'}}, {'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Request a higher quota limit.', 'url': 'https://cloud.google.c

URL 제출 실패 (https://prayer-church.co.kr/november-prayers-2024/): <HttpError 429 when requesting https://indexing.googleapis.com/v3/urlNotifications:publish?alt=json returned "Quota exceeded for quota metric 'Publish requests' and limit 'Publish requests per day' of service 'indexing.googleapis.com' for consumer 'project_number:123230913857'.". Details: "[{'@type': 'type.googleapis.com/google.rpc.ErrorInfo', 'reason': 'RATE_LIMIT_EXCEEDED', 'domain': 'googleapis.com', 'metadata': {'service': 'indexing.googleapis.com', 'quota_limit': 'DefaultPublishRequestsPerDayPerProject', 'quota_unit': '1/d/{project}', 'quota_limit_value': '200', 'consumer': 'projects/123230913857', 'quota_location': 'global', 'quota_metric': 'indexing.googleapis.com/v3_publish_requests'}}, {'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Request a higher quota limit.', 'url': 'https://cloud.google.com/docs/quotas/help/request_increase'}]}]">
URL 제출 실패 (https://prayer-church.co.kr/october-pra

URL 제출 실패 (https://prayer-church.co.kr/february-prayers-2024/): <HttpError 429 when requesting https://indexing.googleapis.com/v3/urlNotifications:publish?alt=json returned "Quota exceeded for quota metric 'Publish requests' and limit 'Publish requests per day' of service 'indexing.googleapis.com' for consumer 'project_number:123230913857'.". Details: "[{'@type': 'type.googleapis.com/google.rpc.ErrorInfo', 'reason': 'RATE_LIMIT_EXCEEDED', 'domain': 'googleapis.com', 'metadata': {'service': 'indexing.googleapis.com', 'consumer': 'projects/123230913857', 'quota_limit': 'DefaultPublishRequestsPerDayPerProject', 'quota_location': 'global', 'quota_limit_value': '200', 'quota_metric': 'indexing.googleapis.com/v3_publish_requests', 'quota_unit': '1/d/{project}'}}, {'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Request a higher quota limit.', 'url': 'https://cloud.google.com/docs/quotas/help/request_increase'}]}]">
URL 제출 실패 (https://prayer-church.co.kr/january-pra

URL 제출 실패 (https://prayer-church.co.kr/june-prayers-2025/): <HttpError 429 when requesting https://indexing.googleapis.com/v3/urlNotifications:publish?alt=json returned "Quota exceeded for quota metric 'Publish requests' and limit 'Publish requests per day' of service 'indexing.googleapis.com' for consumer 'project_number:123230913857'.". Details: "[{'@type': 'type.googleapis.com/google.rpc.ErrorInfo', 'reason': 'RATE_LIMIT_EXCEEDED', 'domain': 'googleapis.com', 'metadata': {'quota_unit': '1/d/{project}', 'quota_metric': 'indexing.googleapis.com/v3_publish_requests', 'consumer': 'projects/123230913857', 'quota_location': 'global', 'quota_limit_value': '200', 'service': 'indexing.googleapis.com', 'quota_limit': 'DefaultPublishRequestsPerDayPerProject'}}, {'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Request a higher quota limit.', 'url': 'https://cloud.google.com/docs/quotas/help/request_increase'}]}]">
URL 제출 실패 (https://prayer-church.co.kr/may-prayers-202

URL 제출 실패 (https://prayer-church.co.kr/prayers-meeting/): <HttpError 429 when requesting https://indexing.googleapis.com/v3/urlNotifications:publish?alt=json returned "Quota exceeded for quota metric 'Publish requests' and limit 'Publish requests per day' of service 'indexing.googleapis.com' for consumer 'project_number:123230913857'.". Details: "[{'@type': 'type.googleapis.com/google.rpc.ErrorInfo', 'reason': 'RATE_LIMIT_EXCEEDED', 'domain': 'googleapis.com', 'metadata': {'quota_unit': '1/d/{project}', 'quota_location': 'global', 'quota_limit': 'DefaultPublishRequestsPerDayPerProject', 'consumer': 'projects/123230913857', 'quota_limit_value': '200', 'quota_metric': 'indexing.googleapis.com/v3_publish_requests', 'service': 'indexing.googleapis.com'}}, {'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Request a higher quota limit.', 'url': 'https://cloud.google.com/docs/quotas/help/request_increase'}]}]">
URL 제출 실패 (https://prayer-church.co.kr/prayers-special-e

URL 제출 실패 (https://prayer-church.co.kr/prayers-for-blessing-children/): <HttpError 429 when requesting https://indexing.googleapis.com/v3/urlNotifications:publish?alt=json returned "Quota exceeded for quota metric 'Publish requests' and limit 'Publish requests per day' of service 'indexing.googleapis.com' for consumer 'project_number:123230913857'.". Details: "[{'@type': 'type.googleapis.com/google.rpc.ErrorInfo', 'reason': 'RATE_LIMIT_EXCEEDED', 'domain': 'googleapis.com', 'metadata': {'quota_location': 'global', 'service': 'indexing.googleapis.com', 'consumer': 'projects/123230913857', 'quota_unit': '1/d/{project}', 'quota_metric': 'indexing.googleapis.com/v3_publish_requests', 'quota_limit_value': '200', 'quota_limit': 'DefaultPublishRequestsPerDayPerProject'}}, {'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Request a higher quota limit.', 'url': 'https://cloud.google.com/docs/quotas/help/request_increase'}]}]">
URL 제출 실패 (https://prayer-church.co.kr/pra

URL 제출 실패 (https://prayer-church.co.kr/wednesday-prayers/): <HttpError 429 when requesting https://indexing.googleapis.com/v3/urlNotifications:publish?alt=json returned "Quota exceeded for quota metric 'Publish requests' and limit 'Publish requests per day' of service 'indexing.googleapis.com' for consumer 'project_number:123230913857'.". Details: "[{'@type': 'type.googleapis.com/google.rpc.ErrorInfo', 'reason': 'RATE_LIMIT_EXCEEDED', 'domain': 'googleapis.com', 'metadata': {'quota_location': 'global', 'quota_limit': 'DefaultPublishRequestsPerDayPerProject', 'quota_unit': '1/d/{project}', 'quota_limit_value': '200', 'service': 'indexing.googleapis.com', 'quota_metric': 'indexing.googleapis.com/v3_publish_requests', 'consumer': 'projects/123230913857'}}, {'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Request a higher quota limit.', 'url': 'https://cloud.google.com/docs/quotas/help/request_increase'}]}]">
URL 제출 실패 (https://prayer-church.co.kr/wednesday-worsh

URL 제출 실패 (https://prayer-church.co.kr/september-prayers/): <HttpError 429 when requesting https://indexing.googleapis.com/v3/urlNotifications:publish?alt=json returned "Quota exceeded for quota metric 'Publish requests' and limit 'Publish requests per day' of service 'indexing.googleapis.com' for consumer 'project_number:123230913857'.". Details: "[{'@type': 'type.googleapis.com/google.rpc.ErrorInfo', 'reason': 'RATE_LIMIT_EXCEEDED', 'domain': 'googleapis.com', 'metadata': {'quota_unit': '1/d/{project}', 'quota_limit_value': '200', 'service': 'indexing.googleapis.com', 'quota_limit': 'DefaultPublishRequestsPerDayPerProject', 'consumer': 'projects/123230913857', 'quota_metric': 'indexing.googleapis.com/v3_publish_requests', 'quota_location': 'global'}}, {'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Request a higher quota limit.', 'url': 'https://cloud.google.com/docs/quotas/help/request_increase'}]}]">
URL 제출 실패 (https://prayer-church.co.kr/august-prayers/

URL 제출 실패 (https://prayer-church.co.kr/monthly-prayer-collection/): <HttpError 429 when requesting https://indexing.googleapis.com/v3/urlNotifications:publish?alt=json returned "Quota exceeded for quota metric 'Publish requests' and limit 'Publish requests per day' of service 'indexing.googleapis.com' for consumer 'project_number:123230913857'.". Details: "[{'@type': 'type.googleapis.com/google.rpc.ErrorInfo', 'reason': 'RATE_LIMIT_EXCEEDED', 'domain': 'googleapis.com', 'metadata': {'quota_metric': 'indexing.googleapis.com/v3_publish_requests', 'service': 'indexing.googleapis.com', 'consumer': 'projects/123230913857', 'quota_unit': '1/d/{project}', 'quota_limit': 'DefaultPublishRequestsPerDayPerProject', 'quota_location': 'global', 'quota_limit_value': '200'}}, {'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Request a higher quota limit.', 'url': 'https://cloud.google.com/docs/quotas/help/request_increase'}]}]">
URL 제출 실패 (https://prayer-church.co.kr/moving-

URL 제출 실패 (https://prayer-church.co.kr/sabbath-worship-prayer/): <HttpError 429 when requesting https://indexing.googleapis.com/v3/urlNotifications:publish?alt=json returned "Quota exceeded for quota metric 'Publish requests' and limit 'Publish requests per day' of service 'indexing.googleapis.com' for consumer 'project_number:123230913857'.". Details: "[{'@type': 'type.googleapis.com/google.rpc.ErrorInfo', 'reason': 'RATE_LIMIT_EXCEEDED', 'domain': 'googleapis.com', 'metadata': {'quota_unit': '1/d/{project}', 'quota_limit_value': '200', 'service': 'indexing.googleapis.com', 'quota_limit': 'DefaultPublishRequestsPerDayPerProject', 'consumer': 'projects/123230913857', 'quota_metric': 'indexing.googleapis.com/v3_publish_requests', 'quota_location': 'global'}}, {'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Request a higher quota limit.', 'url': 'https://cloud.google.com/docs/quotas/help/request_increase'}]}]">
인덱싱 요청 완료: 총 82개 중 0개 성공


In [1]:
########################## 카테고리 페이지 일괄 등록 - 최신순 100개 ##########################

import requests
import json
import time
from google.oauth2 import service_account
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# 워드프레스 설정
WORDPRESS_URL = "https://prayer-church.co.kr"  # 실제 워드프레스 사이트 URL
WORDPRESS_API_URL = f"{WORDPRESS_URL}/wp-json/wp/v2"
ITEMS_PER_REQUEST = 100  # 한 번에 가져올 카테고리 수 (최대 100)

# 구글 인덱싱 API 설정
SERVICE_ACCOUNT_FILE = "/Users/a/Desktop/Work/Blog/2024 Adsense/prompt/black-outlet-426702-n1-98f06232b595.json"  # 서비스 계정 인증 파일

def get_wordpress_categories(page_number=1):
    """워드프레스 API에서 카테고리를 가져옵니다."""
    try:
        response = requests.get(
            f"{WORDPRESS_API_URL}/categories",  # 카테고리 엔드포인트 사용
            params={
                "page": page_number,
                "per_page": ITEMS_PER_REQUEST,
                "_fields": "id,link,name"  # 필요한 필드만 요청
            }
        )
        response.raise_for_status()
        
        # 전체 페이지 수 확인
        total_pages = int(response.headers.get("X-WP-TotalPages", 1))
        
        return response.json(), total_pages
    except requests.exceptions.RequestException as e:
        print(f"워드프레스 API 요청 오류: {e}")
        return [], 0

def get_all_wordpress_categories():
    """모든 워드프레스 카테고리 URL을 수집합니다."""
    all_categories = []
    page_number = 1
    total_page_numbers = 1
    
    while page_number <= total_page_numbers:
        categories, total_page_numbers = get_wordpress_categories(page_number)
        if not categories:
            break
        
        all_categories.extend(categories)
        print(f"카테고리 목록 {page_number}/{total_page_numbers} 처리 완료, {len(categories)}개 카테고리 수집")
        page_number += 1
        time.sleep(1)  # API 호출 제한 방지를 위한 딜레이
    
    return all_categories

def authenticate_indexing_api():
    """구글 Indexing API 인증"""
    try:
        credentials = service_account.Credentials.from_service_account_file(
            SERVICE_ACCOUNT_FILE,
            scopes=['https://www.googleapis.com/auth/indexing']
        )
        service = build('indexing', 'v3', credentials=credentials)
        return service
    except Exception as e:
        print(f"인덱싱 API 인증 오류: {e}")
        return None

def submit_url_to_index(service, url):
    """인덱싱 API에 URL 인덱싱 요청"""
    try:
        response = service.urlNotifications().publish(
            body={
                'url': url,
                'type': 'URL_UPDATED'  # 콘텐츠 업데이트 알림
            }
        ).execute()
        print(f"URL 제출 성공: {url}")
        return True
    except HttpError as e:
        print(f"URL 제출 실패 ({url}): {e}")
        return False

def main():
    # 1. 워드프레스에서 모든 카테고리 가져오기
    print("워드프레스 카테고리 수집 중...")
    categories = get_all_wordpress_categories()
    
    if not categories:
        print("카테고리를 찾을 수 없습니다.")
        return
    
    print(f"총 {len(categories)}개의 카테고리를 찾았습니다.")
    
    # 2. 인덱싱 API 인증
    print("구글 인덱싱 API 인증 중...")
    service = authenticate_indexing_api()
    
    if not service:
        print("인덱싱 API 인증에 실패했습니다.")
        return
    
    # 3. 각 카테고리 URL을 인덱싱 API에 제출
    print("카테고리 URL 인덱싱 제출 시작...")
    success_count = 0
    
    for category in categories:
        category_url = category.get('link')
        category_name = category.get('name')
        if category_url:
            print(f"카테고리 '{category_name}' 인덱싱 요청 중...")
            if submit_url_to_index(service, category_url):
                success_count += 1
            time.sleep(1)  # API 호출 제한 방지를 위한 딜레이
    
    print(f"인덱싱 요청 완료: 총 {len(categories)}개 중 {success_count}개 성공")

if __name__ == "__main__":
    main()

워드프레스 카테고리 수집 중...
카테고리 목록 1/2 처리 완료, 10개 카테고리 수집
카테고리 목록 2/2 처리 완료, 6개 카테고리 수집
총 16개의 카테고리를 찾았습니다.
구글 인덱싱 API 인증 중...
카테고리 URL 인덱싱 제출 시작...
카테고리 '상황별 기도문' 인덱싱 요청 중...
URL 제출 성공: https://prayer-church.co.kr/%ec%83%81%ed%99%a9%eb%b3%84-%ea%b8%b0%eb%8f%84%eb%ac%b8/
카테고리 '새벽예배 대표기도문' 인덱싱 요청 중...
URL 제출 성공: https://prayer-church.co.kr/%ec%83%81%ed%99%a9%eb%b3%84-%ea%b8%b0%eb%8f%84%eb%ac%b8/%ec%83%88%eb%b2%bd%ec%98%88%eb%b0%b0-%eb%8c%80%ed%91%9c%ea%b8%b0%eb%8f%84%eb%ac%b8/
카테고리 '수요예배 대표기도문' 인덱싱 요청 중...
URL 제출 성공: https://prayer-church.co.kr/%ec%88%98%ec%9a%94%ec%98%88%eb%b0%b0-%eb%8c%80%ed%91%9c%ea%b8%b0%eb%8f%84%eb%ac%b8/
카테고리 '쉬운 대표 기도문 모음' 인덱싱 요청 중...
URL 제출 성공: https://prayer-church.co.kr/%ec%89%ac%ec%9a%b4-%eb%8c%80%ed%91%9c-%ea%b8%b0%eb%8f%84%eb%ac%b8-%eb%aa%a8%ec%9d%8c/
카테고리 '심방 기도문' 인덱싱 요청 중...
URL 제출 성공: https://prayer-church.co.kr/%ec%8b%ac%eb%b0%a9-%ea%b8%b0%eb%8f%84%eb%ac%b8/
카테고리 '월별 대표 기도문' 인덱싱 요청 중...
URL 제출 성공: https://prayer-church.co.kr/%ec%9b%94%eb%b3%84-%eb%8c%80%ed%91%9